In [ ]:
# Monto il drive Google

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!pip install codecarbon

# MobileNet-V3 Small + ConvLSTM (soluzione QPT Int con fallback FLoat e Pruning)

Verranno effettuate in ordine le seguenti operazioni:


1.   Caricamento del modello "prunato" al 50 %
2.   Quantizzazione intera 8 bit a fallback Float del modello prunato.

I risultati, sono i seguenti:



*   Peso modello: circa  3.3 MB
*   Accuratezza modello: 80  %
*   Auc: circa 85.5 %
*   Tempo inferenza medio per batch: 0.35 s
*   Consumo Energetico medio per 102 secondi di video:  0.0018 kW/h




In [ ]:
# Carico il modello prunato (non in formato tflite)

from tensorflow.keras.models import load_model


pruned_model= load_model('/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/Pruning/50_perc/pruned_model_ConvLSTM.h5')
pruned_model.summary()

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os

# Dimensioni del batch di input alla rete
batch_size = 16
height = 224
width = 224
channels = 3

# Funzione per estrarre frame da un video e creare i batch
def generate_video_batches(video_paths, seed=None):
    batches = []
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (width, height))
            frames.append(frame)
            if len(frames) == batch_size:
                batches.append(frames)
                frames = []
        cap.release()

    for batch in batches:
        yield [np.array(batch, dtype=np.float32)]

# Imposta il convertitore
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = lambda: generate_video_batches(['/content/gdrive/MyDrive/Dataset/AirtLab-Dataset/Violence/cam1/1.mp4'],
                                                                 seed=42)

# Abilita Select TF ops e disabilita experimental lowering per tensor list ops
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False

tflite_quant_model = converter.convert()

with open('model_quantizzato_prunato_50.tflite', 'wb') as f:
    f.write(tflite_quant_model)

model_quantized_size = os.path.getsize('model_quantizzato_prunato_50.tflite')
model_quantized_size_mb = model_quantized_size / (1024 * 1024)
print(f"Quantizzazione completata e modello salvato come 'model_quantizzato_prunato_50.tflite'  Peso modello: {model_quantized_size_mb} MB")



In [ ]:
# Funzioni per pre processamento dei video e inferenza

import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import time

def load_tflite_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

def preprocess_video(video_path, input_shape, batch_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (input_shape[1], input_shape[0]))
        frame = frame.astype('float32')
        frames.append(frame)

    cap.release()
    frames = np.array(frames)

    if len(frames) >= batch_size:
        num_batches = len(frames) // batch_size
        frames = frames[:num_batches * batch_size]
        batches = np.split(frames, num_batches)
    else:
        batches = []

    return batches

def run_inference(interpreter, input_data):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Calcola il tempo di inferenza
    start_time = time.time()
    interpreter.invoke()
    end_time = time.time()
    inference_time = end_time - start_time

    output_data = interpreter.get_tensor(output_details[0]['index'])

    return output_data, inference_time

def calculate_metrics(y_true, y_pred, y_probs):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    specificity = recall_score(y_true, y_pred, pos_label=0)
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    return accuracy, precision, recall, specificity, roc_auc, fpr, tpr

def plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve'):
    plt.figure()
    plt.plot(fpr, tpr, color='b', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='r', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels


from codecarbon import track_emissions
@track_emissions(project_name="MV3_Small_Inference_ConvLSTM_QPT_intFallbackFLoat_Pruned")
def inferenceV3_ConvLSTM():
    model_path = '/content/model_quantizzato_prunato_50.tflite'
    violence_path = '/content/gdrive/MyDrive/VideoInferenza/Violence'
    nonviolence_path = '/content/gdrive/MyDrive/VideoInferenza/NonViolence'
    input_shape = (224, 224)
    batch_size = 16

    interpreter = load_tflite_model(model_path)

    violence_videos, violence_labels = load_videos_from_folder(violence_path, 1, input_shape, batch_size)
    nonviolence_videos, nonviolence_labels = load_videos_from_folder(nonviolence_path, 0, input_shape, batch_size)

    all_videos = violence_videos + nonviolence_videos
    all_labels = violence_labels + nonviolence_labels

    all_videos = np.array([np.expand_dims(video, axis=0) for video in all_videos])
    all_labels = np.array(all_labels)

    total_inference_time = 0
    y_probs = []
    for video in all_videos:
        output_data, inference_time = run_inference(interpreter, video)
        total_inference_time += inference_time
        y_probs.append(output_data.ravel()[0])

    y_probs = np.array(y_probs)
    y_pred = np.round(y_probs)

    # Calculate metrics
    accuracy, precision, recall, specificity, roc_auc, fpr, tpr = calculate_metrics(all_labels, y_pred, y_probs)

    # Calculate average inference time
    average_inference_time = total_inference_time / len(all_videos)

    # Print metrics
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print(f'Average Inference Time: {average_inference_time:.4f} seconds')

    # Print classification report and confusion matrix
    print('Classification Report:')
    print(classification_report(all_labels, y_pred))

    print('Confusion Matrix:')
    print(confusion_matrix(all_labels, y_pred))

    # Plot ROC curve
    plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve for tf_model_mv3_convLStm_quantizzato_int_fallbackFloat_prunato')

if __name__ == '__main__':
    inferenceV3_ConvLSTM()


# MobileNet-V3 Small + BiLSTM (soluzione QPT Int con fallback FLoat e Pruning)

Verranno effettuate in ordine le seguenti operazioni:


1.   Caricamento del modello "prunato" al 50 %
2.   Quantizzazione intera 8 bit a fallback Float del modello prunato.

I risultati, sono i seguenti:



*   Peso modello: circa  28.9 MB
*   Accuratezza modello: 74.5 %
*   Auc: circa 84.89 %
*   Tempo inferenza medio per batch: 0.415 s
*   Consumo Energetico medio per 102 secondi di video:  0.001784 kW/h


In [ ]:
# Carico il modello prunato (non in formato tflite)
from tensorflow.keras.models import load_model


pruned_model= load_model('/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/Pruning/50_perc/pruned_model_BiLSTM.h5')
pruned_model.summary()

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os

# Dimensione dei batch di input alla rete
batch_size = 16
height = 224
width = 224
channels = 3

# Funzione per estrarre frame da un video e creare i batch
def generate_video_batches(video_paths, seed=None):
    batches = []
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (width, height))
            frames.append(frame)
            if len(frames) == batch_size:
                batches.append(frames)
                frames = []
        cap.release()


    for batch in batches:
        yield [np.array(batch, dtype=np.float32)]

# Imposta il convertitore
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = lambda: generate_video_batches(['/content/gdrive/MyDrive/Dataset/AirtLab-Dataset/Violence/cam1/1.mp4'],
                                                                 seed=42)  # Specifica il seme qui

# Abilita Select TF ops e disabilita experimental lowering per tensor list ops
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False


tflite_quant_model = converter.convert()


with open('model_quantizzato_int_fallbackFloat_prunato_50_BiLSTM.tflite', 'wb') as f:
    f.write(tflite_quant_model)

model_quantized_size = os.path.getsize('model_quantizzato_int_fallbackFloat_prunato_50_BiLSTM.tflite')
model_quantized_size_mb = model_quantized_size / (1024 * 1024)
print(f"Quantizzazione completata e modello salvato come 'model_quantizzato_prunato_50_BiLSTM.tflite'  Peso modello: {model_quantized_size_mb} MB")



In [ ]:
# Funzioni per il pre processamento dei video e l'inferenza

import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import time

def load_tflite_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

def preprocess_video(video_path, input_shape, batch_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (input_shape[1], input_shape[0]))
        frame = frame.astype('float32')
        frames.append(frame)

    cap.release()
    frames = np.array(frames)

    if len(frames) >= batch_size:
        num_batches = len(frames) // batch_size
        frames = frames[:num_batches * batch_size]
        batches = np.split(frames, num_batches)
    else:
        batches = []

    return batches

def run_inference(interpreter, input_data):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Calcola il tempo di inferenza
    start_time = time.time()
    interpreter.invoke()
    end_time = time.time()
    inference_time = end_time - start_time

    output_data = interpreter.get_tensor(output_details[0]['index'])

    return output_data, inference_time

def calculate_metrics(y_true, y_pred, y_probs):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    specificity = recall_score(y_true, y_pred, pos_label=0)
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    return accuracy, precision, recall, specificity, roc_auc, fpr, tpr

def plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve'):
    plt.figure()
    plt.plot(fpr, tpr, color='b', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='r', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels


from codecarbon import track_emissions
@track_emissions(project_name="MV3_Small_Inference_BiLSTM_quantizzato_Int_prunato")
def inferenceV3_BiLSTM():
    model_path = 'model_quantizzato_int_fallbackFloat_prunato_50_BiLSTM.tflite'
    violence_path = '/content/gdrive/MyDrive/VideoInferenza/Violence'
    nonviolence_path = '/content/gdrive/MyDrive/VideoInferenza/NonViolence'
    input_shape = (224, 224)
    batch_size = 16

    interpreter = load_tflite_model(model_path)

    violence_videos, violence_labels = load_videos_from_folder(violence_path, 1, input_shape, batch_size)
    nonviolence_videos, nonviolence_labels = load_videos_from_folder(nonviolence_path, 0, input_shape, batch_size)

    all_videos = violence_videos + nonviolence_videos
    all_labels = violence_labels + nonviolence_labels

    all_videos = np.array([np.expand_dims(video, axis=0) for video in all_videos])
    all_labels = np.array(all_labels)

    total_inference_time = 0
    y_probs = []
    for video in all_videos:
        output_data, inference_time = run_inference(interpreter, video)
        total_inference_time += inference_time
        y_probs.append(output_data.ravel()[0])

    y_probs = np.array(y_probs)
    y_pred = np.round(y_probs)

    # Calculate metrics
    accuracy, precision, recall, specificity, roc_auc, fpr, tpr = calculate_metrics(all_labels, y_pred, y_probs)

    # Calculate average inference time
    average_inference_time = total_inference_time / len(all_videos)

    # Print metrics
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print(f'Average Inference Time: {average_inference_time:.4f} seconds')

    # Print classification report and confusion matrix
    print('Classification Report:')
    print(classification_report(all_labels, y_pred))

    print('Confusion Matrix:')
    print(confusion_matrix(all_labels, y_pred))

    # Plot ROC curve
    plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve for tf_model_mv3_BiLStm_quantizzato_Int_fallback_Float_prunato')

if __name__ == '__main__':
    inferenceV3_BiLSTM()


# MobileNet-V3 Small + ConvLSTM (soluzione QPT  FLoat 16 e Pruning)

Verranno effettuate in ordine le seguenti operazioni:


1.   Caricamento del modello "prunato" al 50 %
2.   Quantizzazione Float16 del modello prunato.


**Attenzione**: Per sfruttare la quantizzazione a FLoat16 è stata utilizzata la Gpu(T4-GPU)

I risultati, sono i seguenti:



*   Peso modello: circa 6 MB
*   Accuratezza modello:  83.98 %
*   Auc: circa 97.87 %
*   Tempo inferenza medio per batch: 0.3 s
*   Consumo Energetico medio per 102 secondi di video: 0.0023  kW/h




In [ ]:
# Carico il modello prunato(non in formato tflite)
from tensorflow.keras.models import load_model


pruned_model= load_model('/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/Pruning/50_perc/pruned_model_ConvLSTM.h5')
pruned_model.summary()

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os

# Dimensioni dei batch di input alla rete neurale
batch_size = 16
height = 224
width = 224
channels = 3

# Funzione per estrarre frame da un video e creare i batch
def generate_video_batches(video_paths, seed=None):
    batches = []
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (width, height))
            frames.append(frame)
            if len(frames) == batch_size:
                batches.append(frames)
                frames = []
        cap.release()

    for batch in batches:
        yield [np.array(batch, dtype=np.float32)]

# Imposta il convertitore
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

# Configurazione della quantizzazione float16
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

# Disabilita la conversione delle operazioni tf.TensorList
converter._experimental_lower_tensor_list_ops = False

tflite_quant_model = converter.convert()

with open('model_quantized_float16_pruned_ConvLSTM.tflite', 'wb') as f:
    f.write(tflite_quant_model)


model_quantized_size = os.path.getsize('model_quantized_float16_pruned_ConvLSTM.tflite')
model_quantized_size_mb = model_quantized_size  / (1024*1024)
print(f"Quantizzazione completata e modello salvato come 'model_quantized_float16_pruned_ConvLSTM.tflite'  , Peso modello: {model_quantized_size_mb} MB")


In [ ]:
# Funzioni di pre processamento dei video e di inferenza

import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import time

def load_tflite_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

def preprocess_video(video_path, input_shape, batch_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (input_shape[1], input_shape[0]))
        frame = frame.astype('float32')
        frames.append(frame)

    cap.release()
    frames = np.array(frames)

    if len(frames) >= batch_size:
        num_batches = len(frames) // batch_size
        frames = frames[:num_batches * batch_size]
        batches = np.split(frames, num_batches)
    else:
        batches = []

    return batches

def run_inference(interpreter, input_data):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Calcola il tempo di inferenza
    start_time = time.time()
    interpreter.invoke()
    end_time = time.time()
    inference_time = end_time - start_time

    output_data = interpreter.get_tensor(output_details[0]['index'])

    return output_data, inference_time

def calculate_metrics(y_true, y_pred, y_probs):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    specificity = recall_score(y_true, y_pred, pos_label=0)
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    return accuracy, precision, recall, specificity, roc_auc, fpr, tpr

def plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve'):
    plt.figure()
    plt.plot(fpr, tpr, color='b', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='r', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels


from codecarbon import track_emissions
@track_emissions(project_name="MV3_Small_Inference_ConvLSTM_quantizzato_Float_pruned")
def inferenceV3_ConvLSTM():
    model_path = 'model_quantized_float16_pruned_ConvLSTM.tflite'
    violence_path = '/content/gdrive/MyDrive/VideoInferenza/Violence'
    nonviolence_path = '/content/gdrive/MyDrive/VideoInferenza/NonViolence'
    input_shape = (224, 224)
    batch_size = 16

    interpreter = load_tflite_model(model_path)

    violence_videos, violence_labels = load_videos_from_folder(violence_path, 1, input_shape, batch_size)
    nonviolence_videos, nonviolence_labels = load_videos_from_folder(nonviolence_path, 0, input_shape, batch_size)

    all_videos = violence_videos + nonviolence_videos
    all_labels = violence_labels + nonviolence_labels

    all_videos = np.array([np.expand_dims(video, axis=0) for video in all_videos])
    all_labels = np.array(all_labels)

    total_inference_time = 0
    y_probs = []
    for video in all_videos:
        output_data, inference_time = run_inference(interpreter, video)
        total_inference_time += inference_time
        y_probs.append(output_data.ravel()[0])

    y_probs = np.array(y_probs)
    y_pred = np.round(y_probs)

    # Calculate metrics
    accuracy, precision, recall, specificity, roc_auc, fpr, tpr = calculate_metrics(all_labels, y_pred, y_probs)

    # Calculate average inference time
    average_inference_time = total_inference_time / len(all_videos)

    # Print metrics
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print(f'Average Inference Time: {average_inference_time:.4f} seconds')

    # Print classification report and confusion matrix
    print('Classification Report:')
    print(classification_report(all_labels, y_pred))

    print('Confusion Matrix:')
    print(confusion_matrix(all_labels, y_pred))

    # Plot ROC curve
    plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve for tf_model_mv3_convLStm_quantizzato_Float16_prunato')

if __name__ == '__main__':
    inferenceV3_ConvLSTM()


# MobileNet-V3 Small + BiLSTM (soluzione QPT  FLoat 16 e Pruning)

Verranno effettuate in ordine le seguenti operazioni:


1.   Caricamento del modello "prunato" al 50 %
2.   Quantizzazione Float16 del modello prunato.


**Attenzione**: Per sfruttare la quantizzazione a FLoat16 è stata utilizzata la Gpu(T4-GPU)

I risultati, sono i seguenti:



*   Peso modello: circa 57.3 MB
*   Accuratezza modello: 69 %
*   Auc: circa 97.5 %
*   Tempo inferenza medio per batch: 0.28 s
*   Consumo Energetico medio per 102 secondi di video: 0.001961  kW/h





In [ ]:
# Carico il modello prunato (non in formato tflite)
from tensorflow.keras.models import load_model


pruned_model= load_model('/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/Pruning/50_perc/pruned_model_BiLSTM.h5')
pruned_model.summary()

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os

# Dimensioni dei batch di input alla rete
batch_size = 16
height = 224
width = 224
channels = 3  # RGB

# Funzione per estrarre frame da un video e creare i batch
def generate_video_batches(video_paths, seed=None):
    batches = []
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (width, height))
            frames.append(frame)
            if len(frames) == batch_size:
                batches.append(frames)
                frames = []
        cap.release()

    for batch in batches:
        yield [np.array(batch, dtype=np.float32)]

# Imposta il convertitore
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

# Configurazione della quantizzazione float16
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

converter._experimental_lower_tensor_list_ops = False

tflite_quant_model = converter.convert()

with open('model_quantized_float16_pruned_BiLSTM.tflite', 'wb') as f:
    f.write(tflite_quant_model)


model_quantized_size = os.path.getsize('model_quantized_float16_pruned_BiLSTM.tflite')
model_quantized_size_mb = model_quantized_size  / (1024*1024)
print(f"Quantizzazione completata e modello salvato come 'model_quantized_float16_pruned_BiLSTM.tflite'  , Peso modello: {model_quantized_size_mb} MB")


In [ ]:
# Funzioni di pre processamento dei video e di inferenza

import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import time

def load_tflite_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

def preprocess_video(video_path, input_shape, batch_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (input_shape[1], input_shape[0]))
        frame = frame.astype('float32')
        frames.append(frame)

    cap.release()
    frames = np.array(frames)

    if len(frames) >= batch_size:
        num_batches = len(frames) // batch_size
        frames = frames[:num_batches * batch_size]
        batches = np.split(frames, num_batches)
    else:
        batches = []

    return batches

def run_inference(interpreter, input_data):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Calcola il tempo di inferenza
    start_time = time.time()
    interpreter.invoke()
    end_time = time.time()
    inference_time = end_time - start_time

    output_data = interpreter.get_tensor(output_details[0]['index'])

    return output_data, inference_time

def calculate_metrics(y_true, y_pred, y_probs):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    specificity = recall_score(y_true, y_pred, pos_label=0)
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    return accuracy, precision, recall, specificity, roc_auc, fpr, tpr

def plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve'):
    plt.figure()
    plt.plot(fpr, tpr, color='b', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='r', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels


from codecarbon import track_emissions
@track_emissions(project_name="MV3_Small_Inference_BiLSTM_quantizzato_Float_pruned")
def inferenceV3_BiLSTM():
    model_path = 'model_quantized_float16_pruned_BiLSTM.tflite'
    violence_path = '/content/gdrive/MyDrive/VideoInferenza/Violence'
    nonviolence_path = '/content/gdrive/MyDrive/VideoInferenza/NonViolence'
    input_shape = (224, 224)
    batch_size = 16

    interpreter = load_tflite_model(model_path)

    violence_videos, violence_labels = load_videos_from_folder(violence_path, 1, input_shape, batch_size)
    nonviolence_videos, nonviolence_labels = load_videos_from_folder(nonviolence_path, 0, input_shape, batch_size)

    all_videos = violence_videos + nonviolence_videos
    all_labels = violence_labels + nonviolence_labels

    all_videos = np.array([np.expand_dims(video, axis=0) for video in all_videos])
    all_labels = np.array(all_labels)

    total_inference_time = 0
    y_probs = []
    for video in all_videos:
        output_data, inference_time = run_inference(interpreter, video)
        total_inference_time += inference_time
        y_probs.append(output_data.ravel()[0])

    y_probs = np.array(y_probs)
    y_pred = np.round(y_probs)

    # Calculate metrics
    accuracy, precision, recall, specificity, roc_auc, fpr, tpr = calculate_metrics(all_labels, y_pred, y_probs)

    # Calculate average inference time
    average_inference_time = total_inference_time / len(all_videos)

    # Print metrics
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print(f'Average Inference Time: {average_inference_time:.4f} seconds')

    # Print classification report and confusion matrix
    print('Classification Report:')
    print(classification_report(all_labels, y_pred))

    print('Confusion Matrix:')
    print(confusion_matrix(all_labels, y_pred))

    # Plot ROC curve
    plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve for tf_model_mv3_BiLStm_quantizzato_Float16_prunato')

if __name__ == '__main__':
    inferenceV3_BiLSTM()
